In [ ]:
# 安裝必要套件
!pip install langgraph openai

In [ ]:
# 導入模組
from langgraph.graph import StateGraph
from dataclasses import dataclass
from openai import OpenAI
from google.colab import userdata

# 定義狀態結構
@dataclass
class TranslateState:
    input: str
    output: str = ""

# 初始化OpenAI客戶端
client = OpenAI(api_key=userdata.get('OPENAI_API_KEY'))  # 替換為您的API金鑰

# 語言模型調用函數
def llm_call(prompt: str) -> str:
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": prompt}]
    )
    return response.choices[0].message.content.strip()

# 翻譯節點：將中文翻譯成英文
def translate_node(state: TranslateState) -> dict:
    prompt = f"將以下中文翻譯成英文：\n{state.input}"
    state.output = llm_call(prompt)
    return {"output": state.output}

# 構建工作流程
workflow = StateGraph(TranslateState)
workflow.add_node("translate", translate_node)
workflow.set_entry_point("translate")
workflow.set_finish_point("translate")
graph = workflow.compile()

# 執行範例
result = graph.invoke(TranslateState(input="人工智慧改變未來"))
print(f"英文翻譯：{result['output']}")